In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error

import numpy as np

In [12]:
# Load the data
train_data = pd.read_csv("./data/train.csv")

# One-hot encode the categorical variables
train_encoded = pd.get_dummies(train_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])

# Separate the target variable and the features
X = train_encoded.drop(columns=["ID", "수요량"])
y = train_encoded["수요량"]

# Apply Standard Scaling
scaler = StandardScaler()
X_standard = scaler.fit_transform(X)

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_standard, y, test_size=0.2, random_state=42)

In [20]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset

# Load the data
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

# One-hot encoding
train_encoded = pd.get_dummies(train_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])
X = train_encoded.drop(columns=["ID", "수요량"]).values
y = train_encoded["수요량"].values

# Splitting data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standard scaling
scaler = StandardScaler()
X_train_standard = scaler.fit_transform(X_train)
X_val_standard = scaler.transform(X_val)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_standard)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_standard)
y_val_tensor = torch.FloatTensor(y_val)

# Neural Network architecture
class NeuralNet(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.layer4(x)
        return x

model = NeuralNet(X_train_tensor.shape[1])

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training with early stopping
num_epochs = 3000
patience = 50
best_val_loss = float('inf')
counter = 0
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor).squeeze()
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor).squeeze()
        val_loss = criterion(val_outputs, y_val_tensor)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        # Save the best model
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

# Load the best model
model.load_state_dict(torch.load("best_model.pth"))

# RMSE on validation set
val_predictions = model(X_val_tensor).squeeze().detach().numpy()
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
print(f"Validation RMSE: {rmse}")

# Preprocessing test data
test_encoded = pd.get_dummies(test_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])
X_test = test_encoded.drop(columns=["ID"]).values
X_test_standard = scaler.transform(X_test)
X_test_tensor = torch.FloatTensor(X_test_standard)

# Predict on test data
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor).squeeze().numpy()

# Submission file
submission_dl = pd.DataFrame({'ID': test_data["ID"], '수요량': test_predictions})
submission_dl.to_csv("./data/submission_dl.csv", index=False)


Epoch 1/3000, Loss: 183284.546875, Val Loss: 190231.96875
Epoch 2/3000, Loss: 183264.546875, Val Loss: 190210.28125
Epoch 3/3000, Loss: 183243.5625, Val Loss: 190187.21875
Epoch 4/3000, Loss: 183221.265625, Val Loss: 190162.359375
Epoch 5/3000, Loss: 183197.265625, Val Loss: 190135.265625
Epoch 6/3000, Loss: 183171.21875, Val Loss: 190105.625
Epoch 7/3000, Loss: 183142.625, Val Loss: 190072.890625
Epoch 8/3000, Loss: 183111.109375, Val Loss: 190036.640625
Epoch 9/3000, Loss: 183076.1875, Val Loss: 189996.1875
Epoch 10/3000, Loss: 183037.28125, Val Loss: 189951.078125
Epoch 11/3000, Loss: 182993.859375, Val Loss: 189900.765625
Epoch 12/3000, Loss: 182945.375, Val Loss: 189844.71875
Epoch 13/3000, Loss: 182891.28125, Val Loss: 189782.40625
Epoch 14/3000, Loss: 182831.03125, Val Loss: 189713.296875
Epoch 15/3000, Loss: 182763.984375, Val Loss: 189636.609375
Epoch 16/3000, Loss: 182689.609375, Val Loss: 189551.578125
Epoch 17/3000, Loss: 182607.140625, Val Loss: 189457.4375
Epoch 18/3000, 

In [22]:
rmse

170.7254120816078

In [24]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load the data
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

# One-hot encoding
train_encoded = pd.get_dummies(train_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])
X = train_encoded.drop(columns=["ID", "수요량"]).values
y = train_encoded["수요량"].values

# Feature Engineering: Add interaction between some features (as an example)
X = np.hstack((X, X[:, 0:1] * X[:, 1:2]))  # Example interaction between first and second feature

# Splitting data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standard scaling
scaler = StandardScaler()
X_train_standard = scaler.fit_transform(X_train)
X_val_standard = scaler.transform(X_val)

# Data Augmentation: Add Gaussian noise
noise_factor = 0.05
X_train_standard += noise_factor * np.random.randn(*X_train_standard.shape)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_standard)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val_standard)
y_val_tensor = torch.FloatTensor(y_val)

# Neural Network architecture with dropout and batch normalization
class NeuralNet(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_dim, 256)
        self.layer2 = nn.Linear(256, 128)
        self.layer3 = nn.Linear(128, 64)
        self.layer4 = nn.Linear(64, 32)
        self.layer5 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.batchnorm1 = nn.BatchNorm1d(256)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.batchnorm4 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = self.relu(self.batchnorm1(self.layer1(x)))
        x = self.dropout(x)
        x = self.relu(self.batchnorm2(self.layer2(x)))
        x = self.dropout(x)
        x = self.relu(self.batchnorm3(self.layer3(x)))
        x = self.dropout(x)
        x = self.relu(self.batchnorm4(self.layer4(x)))
        x = self.layer5(x)
        return x

model = NeuralNet(X_train_tensor.shape[1])

# Loss, optimizer with weight decay, and learning rate scheduler
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

# Training with early stopping and learning rate warm-up
num_epochs = 3000
patience = 200
best_val_loss = float('inf')
counter = 0
warm_up_epochs = 5
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor).squeeze()
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Learning rate warm-up
    if epoch < warm_up_epochs:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001 * (epoch + 1) / warm_up_epochs

    # Validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor).squeeze()
        val_loss = criterion(val_outputs, y_val_tensor)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Early stopping and learning rate reduction on plateau
    scheduler.step(val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

# ...

# Load the best model
model.load_state_dict(torch.load("best_model.pth"))

# RMSE on validation set
val_predictions = model(X_val_tensor).squeeze().detach().numpy()
rmse = np.sqrt(mean_squared_error(y_val, val_predictions))
print(f"Validation RMSE: {rmse}")

# Preprocessing test data
test_encoded = pd.get_dummies(test_data, columns=["쇼핑몰 구분", "도시 유형", "지역 유형", "쇼핑몰 유형", "선물 유형"])

# Feature Engineering for test data (same as train data)
X_test = test_encoded.drop(columns=["ID"]).values
X_test = np.hstack((X_test, X_test[:, 0:1] * X_test[:, 1:2]))  # Example interaction between first and second feature

X_test_standard = scaler.transform(X_test)
X_test_tensor = torch.FloatTensor(X_test_standard)

# Predict on test data
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor).squeeze().numpy()

# Submission file
submission_dl = pd.DataFrame({'ID': test_data["ID"], '수요량': test_predictions})
submission_dl.to_csv("./data/submission_d2.csv", index=False)


Epoch 1/3000, Loss: 183189.140625, Val Loss: 190145.6875
Epoch 2/3000, Loss: 183158.5625, Val Loss: 190147.96875
Epoch 3/3000, Loss: 183148.1875, Val Loss: 190147.828125
Epoch 4/3000, Loss: 183137.84375, Val Loss: 190145.859375
Epoch 5/3000, Loss: 183106.546875, Val Loss: 190141.609375
Epoch 6/3000, Loss: 183077.515625, Val Loss: 190135.0
Epoch 7/3000, Loss: 183036.34375, Val Loss: 190126.046875
Epoch 8/3000, Loss: 182988.015625, Val Loss: 190114.390625
Epoch 9/3000, Loss: 182947.46875, Val Loss: 190099.84375
Epoch 10/3000, Loss: 182899.359375, Val Loss: 190081.125
Epoch 11/3000, Loss: 182847.03125, Val Loss: 190057.15625
Epoch 12/3000, Loss: 182799.71875, Val Loss: 190026.1875
Epoch 13/3000, Loss: 182748.5, Val Loss: 189987.484375
Epoch 14/3000, Loss: 182705.9375, Val Loss: 189939.921875
Epoch 15/3000, Loss: 182658.984375, Val Loss: 189882.640625
Epoch 16/3000, Loss: 182626.15625, Val Loss: 189816.703125
Epoch 17/3000, Loss: 182571.203125, Val Loss: 189742.421875
Epoch 18/3000, Loss: 